In [ ]:
from llama_index.core.index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.embeddings import HuggingFaceEmbedding
import os

# 设置环境变量（可选，取决于你的OpenAI API使用情况）
os.environ["OPENAI_API_KEY"] = "your-api-key"  # 如果使用OpenAI的嵌入模型

def main():
    # 1. 加载文档
    documents = SimpleDirectoryReader("./data").load_data()
    print(f"加载了 {len(documents)} 个文档")
    
    # 2. 使用Sentence-Transformers模型作为嵌入模型
    # 这里使用的是一个多语言模型，特别适合中文
    embed_model = HuggingFaceEmbedding(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )
    
    # 3. 创建向量存储索引
    index = VectorStoreIndex.from_documents(
        documents, 
        embed_model=embed_model,
    )
    
    # 4. 创建查询引擎
    query_engine = index.as_query_engine()
    
    # 5. 执行查询
    query = "人工智能如何应用于医疗领域？"
    response = query_engine.query(query)
    
    print(f"\n查询: {query}")
    print(f"回答: {response}")
    
    # 6. 展示嵌入向量（示例）
    sample_text = "这是一个示例文本，用于展示嵌入向量。"
    embedding = embed_model.get_text_embedding(sample_text)
    
    print(f"\n文本: '{sample_text}'")
    print(f"嵌入维度: {len(embedding)}")
    print(f"嵌入向量前5个值: {embedding[:5]}")
    
    # 7. 相似度比较（示例）
    text1 = "人工智能在医疗诊断中的应用"
    text2 = "AI如何帮助医生进行疾病诊断"
    text3 = "太阳系中的行星"
    
    emb1 = embed_model.get_text_embedding(text1)
    emb2 = embed_model.get_text_embedding(text2)
    emb3 = embed_model.get_text_embedding(text3)
    
    # 计算余弦相似度（简化版）
    import numpy as np
    def cosine_similarity(a, b):
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    sim12 = cosine_similarity(emb1, emb2)
    sim13 = cosine_similarity(emb1, emb3)
    
    print(f"\n相似度比较:")
    print(f"'{text1}' 与 '{text2}' 的相似度: {sim12:.4f}")
    print(f"'{text1}' 与 '{text3}' 的相似度: {sim13:.4f}")
    
    # 输出解释
    print("\n这就是为什么我们需要Sentence-Transformers:")
    print("1. 语义相似的文本（医疗诊断相关）具有较高的向量相似度")
    print("2. 语义不相似的文本（医疗vs行星）具有较低的向量相似度")
    print("3. 这使得我们可以通过向量相似度搜索找到与查询语义最相关的文档")

if __name__ == "__main__":
    main()